In [1]:
import sqlite3
import pandas as pd

testers=['S006','S008','S009','S010','S012','S013','S014','S015','S016','S017',\
         'S018','S019','S020','S021','S022','S023','S024','S025','S026','S027',\
            'S028','S029']

conn=sqlite3.connect('harth.db')

def dataOfTester(tester):
    return 'SELECT * FROM '+tester

def createDataframe(tester):
    tester_df=pd.read_sql_query(dataOfTester(tester),conn)
    if(tester=='S015' or tester=='S021'):
            tester_df=tester_df.drop(columns=['index'])
    if(tester=='S023'):
            tester_df=tester_df.drop(columns=['field1'])
    return tester_df

tester_df = [createDataframe(tester) for tester in testers]

tester_df[0]['timestamp']=pd.to_datetime(tester_df[0]['timestamp'])
interval=pd.Timedelta('0.010s')
tester_df[0]['sample']=(tester_df[0]['timestamp']-tester_df[0]['timestamp'].iloc[0])//interval
tester_df[0].drop(columns=['timestamp'],inplace=True)


# interval=pd.Timedelta('0.010s')
# df_nosens['sample']=(df_nosens['timestamp']-df_nosens['timestamp'].iloc[0])//interval

tester_df[0].head()



,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z,label,sample
0,-0.760242,0.299570,0.468570,-5.092732,-0.298644,0.709439,6,0
1,-0.530138,0.281880,0.319987,0.900547,0.286944,0.340309,6,1
2,-1.170922,0.186353,-0.167010,-0.035442,-0.078423,-0.515212,6,2
3,-0.648772,0.016579,-0.054284,-1.554248,-0.950978,-0.221140,6,3
4,-0.355071,-0.051831,-0.113419,-0.547471,0.140903,-0.653782,6,4


In [2]:
tester_df[0].drop(columns=['sample'],inplace=True)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X=tester_df[0].drop(columns=['label'])
Y=tester_df[0]['label']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=100,random_state=42,n_jobs=4)
rf.fit(X_train,y_train)

RandomForestClassifier(n_jobs=4, random_state=42)

In [4]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

y_pred=rf.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(rf.score(X_train,y_train))

0.9302194710185706
              precision    recall  f1-score   support

           1       0.72      0.81      0.76      7318
           3       0.64      0.41      0.50      3787
           4       0.79      0.27      0.40       434
           5       0.82      0.44      0.58       479
           6       0.92      0.94      0.93     18747
           7       1.00      1.00      1.00     76215
           8       1.00      0.99      1.00      3878
          13       0.72      0.87      0.79      7441
          14       0.61      0.30      0.40       910
         130       0.70      0.58      0.63      3404

    accuracy                           0.93    122613
   macro avg       0.79      0.66      0.70    122613
weighted avg       0.93      0.93      0.93    122613

0.9999965046697612


In [5]:
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(max_iter=1000,activation='tanh',random_state=42,early_stopping=True)
mlp.fit(X_train,y_train)

MLPClassifier(activation='tanh', early_stopping=True, max_iter=1000,
              random_state=42)

In [6]:
from sklearn.metrics import accuracy_score,classification_report
mlppredicts=mlp.predict(X_test)
mlp_report=classification_report(y_test,mlppredicts)
mlp_accur=accuracy_score(y_test,mlppredicts)
mlp_score=mlp.score(X_train,y_train)
print("MLP report:",mlp_report,"MLP accuracy:",mlp_accur,"MLP training:",mlp_score)

MLP report:               precision    recall  f1-score   support

           1       0.68      0.68      0.68      7318
           3       0.47      0.16      0.24      3787
           4       0.49      0.15      0.23       434
           5       0.67      0.42      0.52       479
           6       0.84      0.94      0.88     18747
           7       0.99      1.00      1.00     76215
           8       1.00      0.99      0.99      3878
          13       0.69      0.83      0.75      7441
          14       0.47      0.25      0.32       910
         130       0.62      0.52      0.57      3404

    accuracy                           0.91    122613
   macro avg       0.69      0.59      0.62    122613
weighted avg       0.90      0.91      0.90    122613
 MLP accuracy: 0.9088351153629713 MLP training: 0.9099253397460992


In [9]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [10]:
predicts_gnb=gnb.predict(X_test)
gnb_report=classification_report(y_test,predicts_gnb)
gnb_accur=accuracy_score(y_test,predicts_gnb)
gnb_score=gnb.score(X_train,y_train)
print("GNB estim report: \n",gnb_report,'\n',"GNB estim accuracy:",gnb_accur,'\n',"GNB training:",gnb_score)

GNB estim report: 
               precision    recall  f1-score   support

           1       0.58      0.32      0.41      7318
           3       0.18      0.13      0.15      3787
           4       0.21      0.01      0.01       434
           5       0.09      0.04      0.06       479
           6       0.75      0.94      0.83     18747
           7       1.00      0.99      0.99     76215
           8       1.00      0.95      0.97      3878
          13       0.67      0.69      0.68      7441
          14       0.16      0.18      0.17       910
         130       0.48      0.59      0.53      3404

    accuracy                           0.87    122613
   macro avg       0.51      0.48      0.48    122613
weighted avg       0.86      0.87      0.86    122613
 
 GNB estim accuracy: 0.8716938660664041 
 GNB training: 0.870574911917678
